In [ ]:
!pip3 install torchtext==0.8.1

     |████████████████████████████████| 7.0 MB 3.8 MB/s 
     |████████████████████████████████| 776.8 MB 16 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.7.1 which is incompatible.


In [ ]:
! python3 -m spacy download en

     |████████████████████████████████| 12.0 MB 501 kB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
!pip install msgpack==0.5.6

     |████████████████████████████████| 138 kB 4.0 MB/s 
  Created wheel for msgpack: filename=msgpack-0.5.6-cp37-cp37m-linux_x86_64.whl size=298920 sha256=8a23c9bf42c4c2b179d105b9048dbd919e6bb037659bfbe808739c26f60bc75e
  Stored in directory: /root/.cache/pip/wheels/42/92/5b/708850ed8872c249ce8ace306b542e1099428491e6b613297b
Successfully built msgpack
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.3
    Uninstalling msgpack-1.0.3:
      Successfully uninstalled msgpack-1.0.3


In [ ]:
import torch
from torchtext import data
from torchtext import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

train, test = datasets.IMDB.splits(TEXT, LABEL)

train, valid = train.split(random_state=random.seed(SEED))


/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 12.0MB/s]
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [ ]:
len(train[0].text)

189

In [ ]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train) 

.vector_cache/glove.6B.zip: 862MB [02:43, 5.29MB/s]                           
100%|█████████▉| 399999/400000 [00:16<00:00, 24495.93it/s]


In [ ]:
test_w=TEXT.vocab.itos[9205]
test_w2=TEXT.vocab.itos[9206]

print(test_w)
print(test_w2)

hats
healing


In [ ]:
from torch.nn.functional import cosine_similarity
test_v=TEXT.vocab.vectors[9205].unsqueeze(0)
test_v2=TEXT.vocab.vectors[9206]

cosine_similarity(test_v,TEXT.vocab.vectors,dim=1).sort()


torch.return_types.sort(values=tensor([-0.2809, -0.2443, -0.2430,  ...,  0.7779,  0.7918,  1.0000]), indices=tensor([24894, 10858, 21976,  ...,  9155,  9277,  9205]))

In [ ]:
print(TEXT.vocab.itos[538])

genre


In [ ]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False,
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## Build Model

In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0],embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1],embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2],embedding_dim))
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs,embedding_dim)) for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
# instantiate our CNN class

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.7949, -0.3553, -0.7039,  ...,  1.2660,  0.3886,  0.6810],
        [-0.3033,  1.0977, -0.4089,  ...,  1.5036,  0.1848,  0.4442],
        [-0.6585,  0.7627, -0.4833,  ..., -0.4813, -0.3814,  0.0460]])

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
import torch.nn.functional as F

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y.float()).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [ ]:
def train_model(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label.float())
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate_model(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label.float())
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 5
for epoch in range(N_EPOCHS):

    train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion)
    #valid_loss, valid_acc = evaluate_model(model, valid_iterator, criterion)
    valid_loss=0
    valid_acc=0

    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01, Train Loss: 0.500, Train Acc: 74.31%, Val. Loss: 0.000, Val. Acc: 0.00%
Epoch: 02, Train Loss: 0.307, Train Acc: 87.22%, Val. Loss: 0.000, Val. Acc: 0.00%
Epoch: 03, Train Loss: 0.218, Train Acc: 91.59%, Val. Loss: 0.000, Val. Acc: 0.00%
Epoch: 04, Train Loss: 0.143, Train Acc: 94.83%, Val. Loss: 0.000, Val. Acc: 0.00%
Epoch: 05, Train Loss: 0.092, Train Acc: 96.98%, Val. Loss: 0.000, Val. Acc: 0.00%


In [ ]:
test_loss, test_acc = evaluate_model(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Test Loss: 0.305, Test Acc: 88.38%


In [ ]:
# Some auxiliary functions in order to make a color pallete. 
# Couresy of https://www.oreilly.com/library/view/python-cookbook/0596001673/ch09s11.html

import math

def floatRgb(mag):
    """ Return a tuple of floats between 0 and 1 for R, G, and B. """
    
    #blue  = min((max((4*(0.75-x), 0.)), 1.))
    #red   = min((max((4*(x-0.25), 0.)), 1.))
    #green = min((max((4*math.fabs(x-0.5)-1., 0.)), 1.))
    red=0
    blue=0
    green =0
    
    if mag>0:
      blue=min(1,mag/3)
    else:
      if mag<0:
        red=min(1,-mag/3)
        
        
     
    
    return red, green, blue
  
def rgb(mag):
    """ Return a tuple of integers, as used in AWT/Java plots. """
    red, green, blue = floatRgb(mag)
    return int(red*255), int(green*255), int(blue*255)

def strRgb(mag):
    """ Return a hex string, as used in Tk plots. """
    return "#%02x%02x%02x" % rgb(mag)

In [ ]:
import spacy
import numpy as np
from IPython.core.display import display,HTML
from torch.nn.functional import cosine_similarity

nlp = spacy.load('en')
eps = np.finfo(np.float32).eps.item()
def predict_sentiment(sentence, explain_scores=True,explain_relative_to=1):
  
    tokenized_sentence = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed_sentence = [TEXT.vocab.stoi[t] for t in tokenized_sentence]
    tensor = torch.LongTensor(indexed_sentence).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    
    original_input_embedding,input_grad=get_input_gradients(indexed_sentence,prediction,explain_relative_to)
    
    explanation=get_prediction_explanation(tokenized_sentence,original_input_embedding,input_grad,explain_scores)
      
    return {'tokenized_sentence': tokenized_sentence,'prediction': prediction.item(),'explanation': explanation }

def get_input_gradients(original_sentence,prediction,in_relation_to):
    gradient_truth=torch.Tensor([in_relation_to]).unsqueeze(0)
    if torch.cuda.is_available():
      gradient_truth=gradient_truth.cuda()
    
    loss=criterion(prediction,gradient_truth)
    optimizer.zero_grad()
    loss.backward()
    
    input_grad=torch.Tensor(len(original_sentence),model.embedding.weight.size(1))
    original_input_embedding=torch.Tensor(len(original_sentence),model.embedding.weight.size(1))
    
    for i in range(0,len(original_sentence)):
      original_input_embedding[i]=model.embedding.weight[original_sentence[i]]
      input_grad[i]=model.embedding.weight.grad[original_sentence[i]]
    
    return original_input_embedding,input_grad
    
    

def get_input_scores(input,input_embedding,input_grad):
  
  
  # Take a SGD step using grads
  
  input_after_step=input_embedding-input_grad
  after_grad_norms = torch.norm(input_after_step, 2, 1)
  before_grad_norms = torch.norm(input_embedding, 2, 1)
  variation = after_grad_norms-before_grad_norms
 
  standard_deviation=torch.std(variation)
  mean=torch.mean(variation)
  z_score=(variation-mean)/standard_deviation
 
  return z_score

def old_get_input_scores(input,input_embedding,input_grad):
  
  grad_norms=torch.norm(input_grad,2,1)
  
  return grad_norms/torch.max(grad_norms)

  
def get_prediction_explanation(input,input_embedding,input_grad, explain_scores):
 
  
  input_word_scores=get_input_scores(input,input_embedding,input_grad)
   
  explanation=""
  for i in range(0,len(input)):
    token=input[i]
    token_color=strRgb(input_word_scores[i])
    if explain_scores:
      str_token="%s (%.3f)"%(token,input_word_scores[i])
    else:
      str_token=token
    
    explanation=explanation+'<font color="'+token_color+'">'+str_token+'&nbsp;</font>'
    if i>0 and i%20==0:
      explanation=explanation+"<br/>"
    
  return {'word_scores': input_word_scores,'input_gradient': input_grad,'textual_explanation':explanation }

  
  

In [ ]:
from torch.nn.functional import cosine_similarity

def get_projected_words(word,word_gradient,num_words=1):
  
  word_index=TEXT.vocab.stoi[word]
  word_embedding=TEXT.vocab.vectors[word_index]
  learning_rate=1
  i=0
  result=[]
  
  while i<100000:
    try: 
      word_embedding=word_embedding-learning_rate*word_gradient
    except:
      # We can have a float overflow here if this process gets out of control
      return result
    similarity_value,similarity_index=cosine_similarity(word_embedding.unsqueeze(0),TEXT.vocab.vectors,dim=1).sort(descending=True)
    if similarity_index[0]!=word_index:
      if  similarity_value[0]<0.5:
        break
      
      result.append({'word':TEXT.vocab.itos[similarity_index[0]],'similarity': similarity_value[0]})
      word_index=similarity_index[0]
      learning_rate=1
      if len(result)>=num_words:
        break
      
    i=i+1
    learning_rate=learning_rate*1.1
      
  return result

  
def get_projected_sentence_word(prediction,word):
  
  sentence=prediction['tokenized_sentence']
  word_index_in_sentence=[i for i in range(0,len(sentence)) if sentence[i]==word][0]
  word_gradient=prediction['explanation']['input_gradient'][word_index_in_sentence]
  
  
  return get_projected_words(word,word_gradient,1)
  
  

In [ ]:
prediction=predict_sentiment("This is a ridiculous movie and you should never see it.")
print(prediction['prediction'])
display(HTML(prediction['explanation']['textual_explanation']))
print(get_projected_sentence_word(prediction,'ridiculous'))

0.23816709220409393


[{'word': 'interesting', 'similarity': tensor(0.7005)}]


In [ ]:
prediction=predict_sentiment("Bitcoin sucks ass, divest right now.")
print(prediction['prediction'])
display(HTML(prediction['explanation']['textual_explanation']))
# print(get_projected_sentence_word(prediction,'ridiculous'))

0.19795987010002136
